In this notebook we demonstrate formal hypothesis testing using the NHANES data.

It is important to note that the NHANES data are a "complex survey". The data are not an independent and representative sample from the target population. Proper analysis of complex survey data should make use of additional information about how the data were collected. Since complex survey analysis is a somewhat specialized topic, we ignore this aspect of the data here, and analyze the NHANES data as if it were an independent and identically distributed sample from a population.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy
import scipy.stats.distributions as dist



In [ ]:
df=pd.read_csv(r'/content/drive/MyDrive/NHANES.csv')
df.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [ ]:
df.info

<bound method DataFrame.info of        SEQN  ALQ101  ALQ110  ALQ130  SMQ020  RIAGENDR  RIDAGEYR  RIDRETH1  \
0     83732     1.0     NaN     1.0       1         1        62         3   
1     83733     1.0     NaN     6.0       1         1        53         3   
2     83734     1.0     NaN     NaN       1         1        78         3   
3     83735     2.0     1.0     1.0       2         2        56         3   
4     83736     2.0     1.0     1.0       2         2        42         4   
...     ...     ...     ...     ...     ...       ...       ...       ...   
5730  93695     2.0     2.0     NaN       1         2        76         3   
5731  93696     2.0     2.0     NaN       2         1        26         3   
5732  93697     1.0     NaN     1.0       1         2        80         3   
5733  93700     NaN     NaN     NaN       1         1        35         3   
5734  93702     1.0     NaN     2.0       2         2        24         3   

      DMDCITZN  DMDEDUC2  ...  BPXSY2  BPXD

In [ ]:
df.describe()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
count,5735.000000,5208.000000,1731.000000,3379.000000,5735.000000,5735.000000,5735.000000,5735.000000,5734.000000,5474.000000,...,5535.000000,5535.000000,5666.000000,5673.000000,5662.000000,5345.000000,5427.000000,5427.000000,5368.000000,4732.000000
mean,88678.583435,1.336406,1.587522,3.911512,1.607149,1.518919,48.052310,3.042371,1.185385,3.441725,...,124.783017,69.346703,81.342676,166.142834,29.382197,38.576782,37.146987,33.112235,99.567213,1.915469
std,2882.139237,0.505514,0.623940,34.341839,0.571975,0.499686,18.431011,1.296793,0.491678,1.309700,...,18.527012,13.022829,21.764409,10.079264,7.095921,3.873018,2.800784,5.268027,16.844109,0.414845
min,83732.000000,1.000000,1.000000,1.000000,1.000000,1.000000,18.000000,1.000000,1.000000,1.000000,...,84.000000,0.000000,32.400000,129.700000,14.500000,26.000000,28.200000,17.100000,58.700000,1.000000
25%,86164.000000,1.000000,1.000000,1.000000,1.000000,1.000000,32.000000,2.000000,1.000000,3.000000,...,112.000000,62.000000,65.900000,158.700000,24.300000,36.000000,35.200000,29.500000,87.600000,2.000000
50%,88668.000000,1.000000,2.000000,2.000000,2.000000,2.000000,48.000000,3.000000,1.000000,4.000000,...,122.000000,70.000000,78.200000,166.000000,28.300000,38.600000,37.100000,32.700000,98.300000,2.000000
75%,91178.500000,2.000000,2.000000,3.000000,2.000000,2.000000,63.000000,4.000000,1.000000,4.750000,...,134.000000,78.000000,92.700000,173.500000,33.000000,41.200000,39.000000,36.200000,109.300000,2.000000
max,93702.000000,9.000000,9.000000,999.000000,9.000000,2.000000,80.000000,5.000000,9.000000,9.000000,...,238.000000,144.000000,198.900000,202.700000,67.300000,51.500000,47.400000,58.400000,171.600000,9.000000


Convert integer code to text values 

In [ ]:
df["SMQ020"]

0       1
1       1
2       1
3       2
4       2
       ..
5730    1
5731    2
5732    1
5733    1
5734    2
Name: SMQ020, Length: 5735, dtype: int64

In [ ]:
df["SMQ020x"]=df["SMQ020"].replace({1:"Yes",2:"No"})

In [ ]:
df["SMQ020x"].unique()

array(['Yes', 'No', 7, 9], dtype=object)

In [ ]:
df["SMQ020x"]=df["SMQ020x"].replace({7:np.nan, 9:np.nan})

In [ ]:
df["SMQ020x"].unique()

array(['Yes', 'No', nan], dtype=object)

In [ ]:
df["RIAGENDR"].unique()

array([1, 2])

In [ ]:
df["RIAGENDRx"]=df["RIAGENDR"].replace({1:"Male",2:"Female"})
df["RIAGENDRx"]

0         Male
1         Male
2         Male
3       Female
4       Female
         ...  
5730    Female
5731      Male
5732    Female
5733      Male
5734    Female
Name: RIAGENDRx, Length: 5735, dtype: object

Hypothesis Tests for One Proportion
The most basic hypothesis test may be the one-sample test for a proportion. This test is used if we have specified a particular value as the null value for the proportion, and we wish to assess if the data are compatible with the true parameter value being equal to this specified value. One-sample tests are not used very often in practice, because it is not very common that we have a specific fixed value to use for comparison. For illustration, imagine that the rate of lifetime smoking in another country was known to be 40%, and we wished to assess whether the rate of lifetime smoking in the US were different from 40%. In the following notebook cell, we carry out the (two-sided) one-sample test that the population proportion of smokers is 0.4, and obtain a p-value of 0.43. This indicates that the NHANES data are compatible with the proportion of (ever) smokers in the US being 40%

In [ ]:
x=df.SMQ020x.dropna()=="Yes"

In [ ]:
p=x.mean()

In [ ]:
p

0.4050655021834061

In [ ]:
se=np.sqrt(.4*(1-.4/len(x)))
se

0.632433437131329

In [ ]:
test_stat=(p-0.4)/se
test_stat

0.008009542010275117

In [ ]:
pvalue=2*dist.norm.cdf(-np.abs(test_stat))
print(test_stat,pvalue)

0.008009542010275117 0.9936093784203114


The following cell carries out the same test as performed above using the Statsmodels library. The results in the first (default) case below are slightly different from the results obtained above because Statsmodels by default uses the sample proportion instead of the null proportion when computing the standard error. This distinction is rarely consequential, but we can specify that the null proportion should be used to calculate the standard error, and the results agree exactly with what we calculated above. The first two lines below carry out tests using the normal approximation to the sampling distribution of the test statistic, and the third line below carries uses the exact binomial sampling distribution. We can see here that the p-values are nearly identical in all three cases. This is expected when the sample size is large, and the proportion is not close to either 0 or 1.

In [ ]:
sm.stats.proportions_ztest(x.sum(),len(x),0.4)

(0.7807518954896244, 0.43494843171868214)

In [ ]:
sm.stats.binom_test(x.sum(),len(x),0.4)

0.4340360854459431

**Hypothesis Tests for Two Proportions**
Comparative tests tend to be used much more frequently than tests comparing one population to a fixed value. A two-sample test of proportions is used to assess whether the proportion of individuals with some trait differs between two sub-populations. For example, we can compare the smoking rates between females and males. Since smoking rates vary strongly with age, we do this in the subpopulation of people between 20 and 25 years of age. In the cell below, we carry out this test without using any libraries, implementing all the test procedures covered elsewhere in the course using Python code. We find that the smoking rate for men is around 10 percentage points greater than the smoking rate for females, and this difference is statistically significant (the p-value is around 0.01).

In [ ]:
dx=df[["SMQ020x","RIDAGEYR","RIAGENDRx"]].dropna()
dx.head()

,SMQ020x,RIDAGEYR,RIAGENDRx
0,Yes,62,Male
1,Yes,53,Male
2,Yes,78,Male
3,No,56,Female
4,No,42,Female


In [ ]:
p=dx.groupby("RIAGENDRx")["SMQ020x"].agg([lambda z:np.mean(z=='Yes'),"size"])
p.columns=["Smoke","N"]
print(p)

              Smoke     N
RIAGENDRx                
Female     0.304845  2972
Male       0.513258  2753


Essentially the same test as above can be conducted by converting the "Yes"/"No" responses to numbers (Yes=1, No=0) and conducting a two-sample t-test, as below:

In [ ]:
p_comb=(dx.SMQ020x=="Yes").mean()
va=p_comb *(1-p_comb)
se=np.sqrt(va*(1/p.N.Female+1/p.N.Male))

In [ ]:
(p_comb,va,se)

(0.4050655021834061, 0.2409874411243111, 0.01298546309757376)

In [ ]:
test_stat=(p.Smoke.Female-p.Smoke.Male)/se
p_value=2*dist.norm.cdf(-np.abs(test_stat))
(test_stat,p_value)

(-16.049719603652488, 5.742288777302776e-58)

In [ ]:
dx_females= dx.loc[dx.RIAGENDRx =="Female","SMQ020x"].replace({"Yes":1,"No":0})
dx_females

3       0
4       0
5       0
7       0
12      1
       ..
5724    0
5727    0
5730    1
5732    1
5734    0
Name: SMQ020x, Length: 2972, dtype: int64

In [ ]:
dx_males= dx.loc[dx.RIAGENDRx =="Male","SMQ020x"].replace({"Yes":1,"No":0})
dx_males

0       1
1       1
2       1
6       1
8       0
       ..
5726    1
5728    0
5729    0
5731    0
5733    1
Name: SMQ020x, Length: 2753, dtype: int64

In [ ]:
sm.stats.ttest_ind(dx_females,dx_males)

(-16.42058555898443, 3.032088786691117e-59, 5723.0)

**Hypothesis Tests Comparing Means**
Tests of means are similar in many ways to tests of proportions. Just as with proportions, for comparing means there are one and two-sample tests, z-tests and t-tests, and one-sided and two-sided tests. As with tests of proportions, one-sample tests of means are not very common, but we illustrate a one sample test in the cell below. We compare systolic blood pressure to the fixed value 120 (which is the lower threshold for "pre-hypertension"), and find that the mean is significantly different from 120 (the point estimate of the mean is 126).

In [ ]:
dx=df[["BPXSY1","RIDAGEYR","RIAGENDRx"]].dropna()
dx

,BPXSY1,RIDAGEYR,RIAGENDRx
0,128.0,62,Male
1,146.0,53,Male
2,138.0,78,Male
3,132.0,56,Female
4,100.0,42,Female
...,...,...,...
5730,112.0,76,Female
5731,118.0,26,Male
5732,154.0,80,Female
5733,104.0,35,Male


In [ ]:
dx=dx.loc[(dx.RIDAGEYR>=40)& (dx.RIDAGEYR<=50)&(dx.RIAGENDRx=="Male"),:]
dx

,BPXSY1,RIDAGEYR,RIAGENDRx
10,144.0,46,Male
11,116.0,45,Male
20,110.0,49,Male
42,128.0,42,Male
51,118.0,50,Male
...,...,...,...
5680,134.0,50,Male
5690,138.0,48,Male
5693,96.0,41,Male
5713,116.0,43,Male


In [ ]:
print(dx.BPXSY1.mean())

125.86698337292161


In [ ]:
sm.stats.ztest(dx.BPXSY1,value=120)

(7.469764137102597, 8.033869113167905e-14)

In the cell below, we carry out a formal test of the null hypothesis that the mean blood pressure for women between the ages of 50 and 60 is equal to the mean blood pressure of men between the ages of 50 and 60. The results indicate that while the mean systolic blood pressure for men is slightly greater than that for women (129 mm/Hg versus 128 mm/Hg), this difference is not statistically significant.

There are a number of different variants on the two-sample t-test. Two often-encountered variants are the t-test carried out using the t-distribution, and the t-test carried out using the normal approximation to the reference distribution of the test statistic, often called a z-test. Below we display results from both these testing approaches. When the sample size is large, the difference between the t-test and z-test is very small.

In [ ]:
dx=df[["BPXSY1","RIDAGEYR","RIAGENDRx"]].dropna()
dx=dx.loc[(dx.RIDAGEYR>=50)& (dx.RIDAGEYR <=60),:]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female


In [ ]:
bpx_female=dx.loc[dx.RIAGENDRx=='Female','BPXSY1']
bpx_female


3       132.0
15      134.0
19      136.0
23      116.0
27      142.0
        ...  
5648    146.0
5661    114.0
5675    136.0
5681    124.0
5686    132.0
Name: BPXSY1, Length: 484, dtype: float64

In [ ]:
bpx_male=dx.loc[dx.RIAGENDRx=='Male','BPXSY1']
print(bpx_female.mean(),bpx_male.mean())

127.92561983471074 129.23829787234044


In [ ]:
print(sm.stats.ztest(bpx_female,bpx_male))

(-1.105435895556249, 0.2689707570859362)


In [ ]:
print(sm.stats.ttest_ind(bpx_female,bpx_male))

(-1.105435895556249, 0.26925004137768577, 952.0)


Another important aspect of two-sample mean testing is "heteroscedasticity", meaning that the variances within the two groups being compared may be different. While the goal of the test is to compare the means, the variances play an important role in calibrating the statistics (deciding how big the mean difference needs to be to be declared statisitically significant). In the NHANES data, we see that there are moderate differences between the amount of variation in BMI for females and for males, looking within 10-year age bands. In every age band, females having greater variation than males.

In [ ]:
dx=df[["BMXBMI","RIDAGEYR","RIAGENDRx"]].dropna()
df["agegrp"]=pd.cut(df.RIDAGEYR,[18,30,40,50,60,70,80])
df.groupby(["agegrp","RIAGENDRx"])["BMXBMI"].agg(np.std).unstack()

RIAGENDRx,Female,Male
agegrp,,
"(18, 30]",7.745893,6.649440
"(30, 40]",8.315608,6.622412
"(40, 50]",8.076195,6.407076
"(50, 60]",7.575848,5.914373
"(60, 70]",7.604514,5.933307
"(70, 80]",6.284968,4.974855


The standard error of the mean difference (e.g. mean female blood pressure minus mean mal blood pressure) can be estimated in at least two different ways. In the statsmodels library, these approaches are referred to as the "pooled" and the "unequal" approach to estimating the variance. If the variances are equal (i.e. there is no heteroscedasticity), then there should be little difference between the two approaches. Even in the presence of moderate heteroscedasticity, as we have here, we can see that the results for the two differences are quite similar. Below we have a loop that considers each 10-year age band and assesses the evidence for a difference in mean BMI for women and for men. The results printed in each row of output are the test-statistic and p-value.

In [ ]:
for k, v in df.groupby("agegrp"):
  bmi_female=v.loc[v.RIAGENDRx=='Female',"BMXBMI"].dropna()
  bmi_female=sm.stats.DescrStatsW(bmi_female)
  bmi_male=v.loc[v.RIAGENDRx=='Male',"BMXBMI"].dropna()
  bmi_male=sm.stats.DescrStatsW(bmi_male)
  print(k)
  print("pooled: ",sm.stats.CompareMeans(bmi_female,bmi_male).ztest_ind(usevar='pooled'))
  print("unequal: ",sm.stats.CompareMeans(bmi_female,bmi_male).ztest_ind(usevar='unequal'))
  print()

(18, 30]
pooled:  (1.7026932933643388, 0.08862548061449649)
unequal:  (1.7174610823927268, 0.08589495934713022)

(30, 40]
pooled:  (1.4378280405644916, 0.1504828511464818)
unequal:  (1.4437869620833494, 0.14879891057892475)

(40, 50]
pooled:  (2.8933761158070186, 0.003811246059501354)
unequal:  (2.9678691663536725, 0.0029987194174035366)

(50, 60]
pooled:  (3.362108779981367, 0.0007734964571391746)
unequal:  (3.375494390173923, 0.0007368319423226574)

(60, 70]
pooled:  (3.6172401442432753, 0.000297761021031936)
unequal:  (3.62848309454456, 0.0002850914147149227)

(70, 80]
pooled:  (2.926729252512258, 0.0034254694144858636)
unequal:  (2.937779886769224, 0.003305716331519299)

